Author: Chris J. Dallmann \
Affiliation: University of Wuerzburg \
Last revision: 12-March-2025

In [4]:
from neuprint import Client
from neuprint import fetch_neurons
from neuprint import fetch_adjacencies
from neuprint import NeuronCriteria as NC

In [5]:
# Select whether to analyze chief 9A or web connectivity
source_name = 'web' # 'web' # 'chief_9A', 'web'

# For chief 9A neurons, select which specific neuron to analyze. For web, set source_index to 0. 
source_index = 0 # 0, 1, 2, 3, 4, 5 for chief 9A neurons in L1, L2, L3, R1, R2, R3

In [18]:
# Initialize client
# token = '' # From https://neuprint.janelia.org/account
c = Client('neuprint.janelia.org', dataset='manc:v1.0', token=token)
c.fetch_version()

In [8]:
# Set neuron IDs
dict_ids = {}
dict_ids['chief_9A'] = [100513, 13157, 14517, 165560, 12443, 12804] # L1, L2, L3, R1, R2, R3
dict_ids['web'] = [10107] # Left VNC

# Add IDs of hook neurons
criteria = NC(type='SNpp38')
df_neuron, _= fetch_neurons(criteria)
dict_ids['hook'] = df_neuron.bodyId.to_list()

In [9]:
# Set source ID
source_id = dict_ids[source_name][source_index]

In [10]:
# Find postsynaptic neurons
df_neuron, df_connectivity = fetch_adjacencies(source_id, None)

In [11]:
# Sum weights across neuropils and sort
df_connectivity = df_connectivity.groupby(['bodyId_pre','bodyId_post'])['weight'].sum().reset_index().sort_values('weight', ascending=False)

In [12]:
# Remove connections less than weight_thresh
weight_thresh = 5 
df_connectivity = df_connectivity[df_connectivity.weight >= weight_thresh]

In [13]:
# Find type of partner neurons  
neuron_type = [] 
n_syn_specific_neurons = 0
for index, row in df_connectivity.iterrows():
    body_id = df_connectivity.bodyId_post[index]
    n_syn = df_connectivity.weight[index].item()
    neuron_type_temp = df_neuron.type[df_neuron.bodyId == body_id].item()
    
    if neuron_type_temp is None:
        neuron_type.append('Unknown')
    elif 'IN' in neuron_type_temp:
        neuron_type.append('IN')
        # For web, get connectivity with chief 9As
        if (source_name == 'web') & (body_id in dict_ids['chief_9A']):
            n_syn_specific_neurons += n_syn
    elif 'MN' in neuron_type_temp:
        neuron_type.append('MN')
    elif 'AN' in neuron_type_temp:
        neuron_type.append('AN')
    elif 'DN' in neuron_type_temp:
        neuron_type.append('DN')
    elif 'SN' in neuron_type_temp or 'SA' in neuron_type_temp:
        neuron_type.append('SN')
        # For chief 9A, get connectivity with hook neurons
        if (source_name == 'chief_9A') & (body_id in dict_ids['hook']):
            n_syn_specific_neurons += n_syn
    elif 'EN' in neuron_type_temp or 'EA' in neuron_type_temp:
        # The web neuron makes only few synapses with efferent neurons (ENs). They are counted as IN here.
        neuron_type.append('IN') 
        
df_connectivity['neuron_type'] = neuron_type

In [14]:
# Sum weights for each type 
df_connectivity_summed = df_connectivity.groupby(['neuron_type'])['weight'].sum().reset_index().sort_values('weight', ascending=False)
df_connectivity_summed

,neuron_type,weight
2,IN,10772
4,Unknown,958
3,MN,894
0,AN,702
1,DN,247


In [16]:
# Calculate relative weight for a neuron type 
neuron_type = 'IN'
int(df_connectivity_summed.weight[df_connectivity_summed.neuron_type==neuron_type].item()) / int(df_connectivity_summed.weight.sum())

0.7936344212775363

In [17]:
# Calculate relative weight for specific neurons 
n_syn_specific_neurons / int(df_connectivity_summed.weight.sum())

0.01930302807043395